how this notebook works

In [ ]:
# dependencies

import pandas as pd
import sqlalchemy as sq
import sys, os
import pickle
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.ensemble import (
    BalancedBaggingClassifier,
    EasyEnsembleClassifier,
)

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report
)

sys.path.append('../')
os.chdir('../')
from ModelBuilderMethods import getConn, extractYears, scaleColumns, encodeColumns
from Models.models_ensemble import getBalancedClassifier, getClassifier

In [ ]:
# unlimited line output
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', 500)

setting up a dataset

In [ ]:
weatherStationQuery = sq.text("""
    SELECT * from dataset_cross_monthly_station
""")

weatherSatQuery = sq.text("""
    SELECT * from dataset_cross_monthly_sat
""")

ergotPrevYearsAggQuery = sq.text("""
    SELECT year, district, 
    present_prev1, present_prev2, present_prev3,
    percnt_true_prev1, percnt_true_prev2, percnt_true_prev3 
    from agg_ergot_sample_v2
""")

ergotTargetQuery = sq.text("""
    SELECT year, district, downgrade from ergot_sample_feat_eng
""")

In [ ]:
conn = getConn("./.env")

stationDf = pd.read_sql(weatherStationQuery, conn)
# satelliteDf = pd.read_sql(weatherSatQuery, conn)
ergotPrevDf = pd.read_sql(ergotPrevYearsAggQuery, conn)
ergotTargetDf = pd.read_sql(ergotTargetQuery, conn)

conn.close()
del conn

In [ ]:
# merge on year and district
# tempdf = pd.merge(satelliteDf, ergotPrevDf, on=["year", "district"], how="left")
# del satelliteDf
# del ergotPrevDf
# tempdf = satelliteDf
tempdf = stationDf

# merge on year and district
datasetDf = pd.merge(ergotTargetDf, tempdf, on=["year", "district"], how="left")
del ergotTargetDf
del tempdf

splitting the dataset

In [ ]:
# train 1995 - 2015 test 2016 - 2020
trainDf = extractYears(datasetDf, 1995, 2015)
testDf = extractYears(datasetDf, 2016, 2020)
del datasetDf

balancing the dataset https://imbalanced-learn.org/stable/



In [ ]:
# pre balancing check
# print value counts downgrade
print(trainDf["downgrade"].value_counts())
print(testDf["downgrade"].value_counts())

In [ ]:
# count nan
print(trainDf.isna().sum())
# set nan to 0
trainDf = trainDf.fillna(0)

In [ ]:
balancer = SMOTEENN(sampling_strategy=0.8, random_state=42)
balancedTrainDfX, balancedTrainDfY = balancer.fit_resample(trainDf.drop(columns="downgrade"), trainDf["downgrade"])

In [ ]:
# post balancing check
# print value counts downgrade
print(balancedTrainDfY.value_counts())

### normalization / scaling
some blurb about scalers  
0 [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)             
1 [MaxAbsScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html)  
2 [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)  
3 [RobustScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)  
4 [Normalizer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html)  
5 [PowerTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html)  
6 [QuantileTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html)  

In [ ]:
# df = pd.DataFrame()
# scaled = scaleColumns(df, ['max_temp'], None, 1)

categorical values [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)  


In [ ]:
# encoded = encodeColumns(df, ['max_temp'], None)

In [ ]:
def printMetrics(model_name, y_true, y_pred):
    print(model_name)
    print("Accuracy: ", accuracy_score(y_true, y_pred))
    print("Precision: ", precision_score(y_true, y_pred))
    print("Recall: ", recall_score(y_true, y_pred))
    print("F1: ", f1_score(y_true, y_pred))
    print("ROC AUC: ", roc_auc_score(y_true, y_pred))
    print("Classification Report: \n", classification_report(y_true, y_pred))
    print()

selecting a model

In [ ]:
ESTIMATORS = 400
DEPTH = 40
CORES = -1
MINSPLSPLIT = 8
MINSAMPLELEAF = 4

model_rf = RandomForestClassifier(
    n_estimators=ESTIMATORS,
    random_state=42,
    max_depth=DEPTH,
    n_jobs=CORES,
    min_samples_split=MINSPLSPLIT,
    min_samples_leaf=MINSAMPLELEAF,
)
model_nobalance_rf = RandomForestClassifier(
    n_estimators=ESTIMATORS,
    random_state=42,
    max_depth=DEPTH,
    n_jobs=CORES,
    min_samples_split=MINSPLSPLIT,
    min_samples_leaf=MINSAMPLELEAF,
)
balanced_model_rf = BalancedRandomForestClassifier(
    n_estimators=ESTIMATORS,
    random_state=42,
    max_depth=DEPTH,
    n_jobs=CORES,
    min_samples_split=MINSPLSPLIT,
    min_samples_leaf=MINSAMPLELEAF,
)
balanced_model_balanced_rf = BalancedRandomForestClassifier(
    n_estimators=ESTIMATORS,
    random_state=42,
    max_depth=DEPTH,
    n_jobs=CORES,
    min_samples_split=MINSPLSPLIT,
    min_samples_leaf=MINSAMPLELEAF,
)
gradient_boosting_model = GradientBoostingClassifier(
    n_estimators=ESTIMATORS, random_state=42, max_depth=DEPTH, verbose=1, n_iter_no_change=100
)
balanced_gradient_boosting_model = GradientBoostingClassifier(
    n_estimators=ESTIMATORS, random_state=42, max_depth=DEPTH, verbose=1, n_iter_no_change=100
)

In [ ]:
model_rf.fit(balancedTrainDfX, balancedTrainDfY)
model_nobalance_rf.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
balanced_model_rf.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
balanced_model_balanced_rf.fit(balancedTrainDfX, balancedTrainDfY)
gradient_boosting_model.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
balanced_gradient_boosting_model.fit(balancedTrainDfX, balancedTrainDfY)

eval procedure

In [ ]:
# set nan to 0
testDf = testDf.fillna(0)

In [ ]:

# get predictions
predictions = model_rf.predict(testDf.drop(columns="downgrade"))
predictions_nobalance = model_nobalance_rf.predict(testDf.drop(columns="downgrade"))
predictions_balanced = balanced_model_rf.predict(testDf.drop(columns="downgrade"))
predictions_balanced_balanced = balanced_model_balanced_rf.predict(testDf.drop(columns="downgrade"))
predictions_gradient_boosting = gradient_boosting_model.predict(testDf.drop(columns="downgrade"))
predictions_balanced_gradient_boosting = balanced_gradient_boosting_model.predict(testDf.drop(columns="downgrade"))


In [ ]:
print(pd.DataFrame(predictions).value_counts())
print(pd.DataFrame(predictions_nobalance).value_counts())
print(pd.DataFrame(predictions_balanced).value_counts())
print(pd.DataFrame(predictions_balanced_balanced).value_counts())
print(pd.DataFrame(predictions_gradient_boosting).value_counts())
print(pd.DataFrame(predictions_balanced_gradient_boosting).value_counts())

In [ ]:

# get accuracy precision recall f1 roc_auc
printMetrics("sk GB imbalanced train set", testDf["downgrade"], predictions_gradient_boosting)
printMetrics("imb GB balanced train set", testDf["downgrade"], predictions_balanced_gradient_boosting)

print model performance metrics on test data

In [ ]:

printMetrics("sk RF balanced train set", testDf["downgrade"], predictions)
printMetrics("sk RF imbalanced train set", testDf["downgrade"], predictions_nobalance)
printMetrics("imb RF imbalanced train set", testDf["downgrade"], predictions_balanced)
printMetrics("imb RF balanced train set", testDf["downgrade"], predictions_balanced_balanced)